<a href="https://colab.research.google.com/github/pragatisachan/ProjectChalenge1/blob/main/spark_corr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=90b3595216b9422887c2a72e8c6d53d50e6b3e074a4f9f33624ab8e426b58b0a
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [57]:
import pyspark


In [3]:
from pyspark.sql import SparkSession


In [6]:
Spark = SparkSession.builder.appName('Practice').getOrCreate()


In [7]:
df = Spark.read.csv("/content/card_transdata.csv" )


In [8]:
df = Spark.read.option('header' , 'true').csv("/content/card_transdata.csv").show()
df


+------------------+------------------------------+------------------------------+---------------+---------+---------------+------------+-----+
|distance_from_home|distance_from_last_transaction|ratio_to_median_purchase_price|repeat_retailer|used_chip|used_pin_number|online_order|fraud|
+------------------+------------------------------+------------------------------+---------------+---------+---------------+------------+-----+
| 57.87785658389723|            0.3111400080477545|            1.9459399775518593|            1.0|      1.0|            0.0|         0.0|  0.0|
|10.829942699255545|           0.17559150228166587|            1.2942188106198573|            1.0|      0.0|            0.0|         0.0|  0.0|
| 5.091079490616996|            0.8051525945853258|           0.42771456119427587|            1.0|      0.0|            0.0|         1.0|  0.0|
|2.2475643282963613|              5.60004354707232|           0.36266257805709584|            1.0|      1.0|            0.0|         1.0

In [9]:
df = Spark.read.csv('/content/card_transdata.csv' , header = True , inferSchema=True)
df.show()


+------------------+------------------------------+------------------------------+---------------+---------+---------------+------------+-----+
|distance_from_home|distance_from_last_transaction|ratio_to_median_purchase_price|repeat_retailer|used_chip|used_pin_number|online_order|fraud|
+------------------+------------------------------+------------------------------+---------------+---------+---------------+------------+-----+
| 57.87785658389723|            0.3111400080477545|            1.9459399775518593|            1.0|      1.0|            0.0|         0.0|  0.0|
|10.829942699255545|           0.17559150228166587|            1.2942188106198573|            1.0|      0.0|            0.0|         0.0|  0.0|
| 5.091079490616996|            0.8051525945853258|           0.42771456119427587|            1.0|      0.0|            0.0|         1.0|  0.0|
|2.2475643282963613|              5.60004354707232|           0.36266257805709584|            1.0|      1.0|            0.0|         1.0

In [10]:
df.printSchema()


root
 |-- distance_from_home: double (nullable = true)
 |-- distance_from_last_transaction: double (nullable = true)
 |-- ratio_to_median_purchase_price: double (nullable = true)
 |-- repeat_retailer: double (nullable = true)
 |-- used_chip: double (nullable = true)
 |-- used_pin_number: double (nullable = true)
 |-- online_order: double (nullable = true)
 |-- fraud: double (nullable = true)



In [11]:
df.columns


['distance_from_home',
 'distance_from_last_transaction',
 'ratio_to_median_purchase_price',
 'repeat_retailer',
 'used_chip',
 'used_pin_number',
 'online_order',
 'fraud']

In [31]:
# corrleation = df.corr()
# corrleation


In [13]:
df.corr("distance_from_home" , "fraud")


0.18790779940319666

In [14]:
df.corr("distance_from_last_transaction" , "fraud")


0.09878003733841226

In [15]:
df.corr("ratio_to_median_purchase_price" , "fraud")


0.46184521796140604

In [16]:
df.corr("repeat_retailer" , "fraud")


-0.0011475367246153887

In [17]:
df.corr("used_chip" , "fraud")

-0.06115005419846402

In [18]:
df.corr("used_pin_number" , "fraud")


-0.10046176050170984

In [19]:
df.corr("online_order" , "fraud")


0.19150035064811433

In [20]:
df.corr("fraud" , "fraud")


1.0

In [21]:
pip install matplotlib seaborn pyspark


In [32]:
from pyspark.ml.feature import VectorAssembler


In [33]:
vector_col = "corr_features"


In [34]:
df.columns


['distance_from_home',
 'distance_from_last_transaction',
 'ratio_to_median_purchase_price',
 'repeat_retailer',
 'used_chip',
 'used_pin_number',
 'online_order',
 'fraud']

In [35]:
assembler = VectorAssembler(inputCols=df.columns , outputCol= vector_col)


In [36]:
df_vector = assembler.transform(df).select(vector_col)


In [37]:
df_vector


DataFrame[corr_features: vector]

In [38]:
from pyspark.ml.stat import Correlation


In [39]:
matrix = Correlation.corr(df_vector , vector_col).collect()[0][0]


In [40]:
corrmatrix = matrix.toArray().tolist()


In [41]:
corrmatrix


[[1.0,
  -0.0012498216754251639,
  -0.001788013343586249,
  0.14613860017659266,
  0.0016185184843323214,
  -0.0011071333052813123,
  -0.003048934569343122,
  0.18790779940329572],
 [-0.0012498216754251639,
  1.0,
  -0.0007208383509684481,
  -0.005791350582480571,
  0.0015539687759239566,
  0.000927747142882833,
  0.000489441663753008,
  0.09878002875482161],
 [-0.001788013343586249,
  -0.0007208383509684481,
  1.0,
  -0.00038002940707719264,
  0.0009045049837426442,
  -0.0011962907070828767,
  0.0012979511134928426,
  0.46184519419984815],
 [0.14613860017659266,
  -0.005791350582480571,
  -0.00038002940707719264,
  1.0,
  -0.002943039290438686,
  -0.002160677069601752,
  -0.0014543038331442353,
  -0.0011518838119935662],
 [0.0016185184843323214,
  0.0015539687759239566,
  0.0009045049837426442,
  -0.002943039290438686,
  1.0,
  -0.0006201706083193151,
  0.00157598883037557,
  -0.061150054198518204],
 [-0.0011071333052813123,
  0.000927747142882833,
  -0.0011962907070828767,
  -0.00216

In [42]:
df.columns


['distance_from_home',
 'distance_from_last_transaction',
 'ratio_to_median_purchase_price',
 'repeat_retailer',
 'used_chip',
 'used_pin_number',
 'online_order',
 'fraud']

In [43]:
columns = ['distance_from_home',
 'distance_from_last_transaction',
 'ratio_to_median_purchase_price',
 'repeat_retailer',
 'used_chip',
 'used_pin_number',
 'online_order',
 'fraud']


In [44]:
df_corr = Spark.createDataFrame(corrmatrix , columns)


In [45]:
df_corr.show()


+--------------------+------------------------------+------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  distance_from_home|distance_from_last_transaction|ratio_to_median_purchase_price|     repeat_retailer|           used_chip|     used_pin_number|        online_order|               fraud|
+--------------------+------------------------------+------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                 1.0|          -0.00124982167542...|          -0.00178801334358...| 0.14613860017659266|0.001618518484332...|-0.00110713330528...|-0.00304893456934...| 0.18790779940329572|
|-0.00124982167542...|                           1.0|          -7.20838350968448...|-0.00579135058248...|0.001553968775923...| 9.27747142882833E-4| 4.89441663753008E-4| 0.09878002875482161|
|-0.00178801334358...|          -7.20838350968448.

In [46]:
df.columns


['distance_from_home',
 'distance_from_last_transaction',
 'ratio_to_median_purchase_price',
 'repeat_retailer',
 'used_chip',
 'used_pin_number',
 'online_order',
 'fraud']

In [52]:
df_corr['distance_from_home','distance_from_last_transaction','fraud'].show()


+--------------------+------------------------------+--------------------+
|  distance_from_home|distance_from_last_transaction|               fraud|
+--------------------+------------------------------+--------------------+
|                 1.0|          -0.00124982167542...| 0.18790779940329572|
|-0.00124982167542...|                           1.0| 0.09878002875482161|
|-0.00178801334358...|          -7.20838350968448...| 0.46184519419984815|
| 0.14613860017659266|          -0.00579135058248...|-0.00115188381199...|
|0.001618518484332...|          0.001553968775923...|-0.06115005419851...|
|-0.00110713330528...|           9.27747142882833E-4|-0.10046176050171836|
|-0.00304893456934...|           4.89441663753008E-4| 0.19150035064821175|
| 0.18790779940329572|           0.09878002875482161|                 1.0|
+--------------------+------------------------------+--------------------+



In [54]:
df_corr.select('distance_from_home','distance_from_last_transaction','fraud').show()


+--------------------+------------------------------+--------------------+
|  distance_from_home|distance_from_last_transaction|               fraud|
+--------------------+------------------------------+--------------------+
|                 1.0|          -0.00124982167542...| 0.18790779940329572|
|-0.00124982167542...|                           1.0| 0.09878002875482161|
|-0.00178801334358...|          -7.20838350968448...| 0.46184519419984815|
| 0.14613860017659266|          -0.00579135058248...|-0.00115188381199...|
|0.001618518484332...|          0.001553968775923...|-0.06115005419851...|
|-0.00110713330528...|           9.27747142882833E-4|-0.10046176050171836|
|-0.00304893456934...|           4.89441663753008E-4| 0.19150035064821175|
| 0.18790779940329572|           0.09878002875482161|                 1.0|
+--------------------+------------------------------+--------------------+

